In [150]:
import pickle as pkl
import pandas as pd

# Imputer
with open('../model/imputer.pkl','rb') as fichier_pickler:
    imputer = pkl.load(fichier_pickler)


# Classification
with open('../model/modele_classification.pkl','rb') as fichier_pickler:
    model_classif = pkl.load(fichier_pickler)



# Regression
with open('../model/modele_regression_code_1.pkl','rb') as fichier_pickler:
    model_reg_1 = pkl.load(fichier_pickler)

with open('../model/modele_regression_code_2.pkl','rb') as fichier_pickler:
    model_reg_2 = pkl.load(fichier_pickler)

with open('../model/modele_regression_code_3.pkl','rb') as fichier_pickler:
    model_reg_3 = pkl.load(fichier_pickler)

with open('../model/modele_regression_code_4.pkl','rb') as fichier_pickler:
    model_reg_4 = pkl.load(fichier_pickler)



In [151]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer



# colonnes_a_imputer = ["Surface reelle bati","Nombre pieces principales"]

# transformateur_medianne = ColumnTransformer(
#     transformers=[('imputer',imputer,colonnes_a_imputer)]
# )

# # Pipeline classif
# pipeline_classif = Pipeline([
#     ('median_imputer',transformateur_medianne),
#     ('classifier',model_classif)
# ])

# # Pipeline Type Local == 1
# pipeline_reg_1 = Pipeline([
#     ('reg_1',model_reg_1)
# ])

# pipeline_reg_2 = Pipeline([
#     ('reg_2',model_reg_2)
# ])

# pipeline_reg_3 = Pipeline([
#     ('reg_3',model_reg_3)
# ])

# pipeline_reg_4 = Pipeline([
#     ('reg_4',model_reg_4)
# ])

In [152]:
#
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:.2f}'.format)


sub = pd.read_csv('../submissions.csv')
sub.columns = sub.columns.str.replace(' ', '_')

sub.head()

C:\Users\nagrimault\AppData\Local\Temp\ipykernel_3332\656433933.py:6: DtypeWarning: Columns (9,15,17,19,31) have mixed types. Specify dtype option on import or set low_memory=False.
  sub = pd.read_csv('../submissions.csv')


,Date_mutation,Nature_mutation,No_voie,B/T/Q,Type_de_voie,Code_voie,Voie,Code_postal,Commune,Code_departement,Code_commune,Prefixe_de_section,Section,No_plan,No_Volume,1er_lot,Surface_Carrez_du_1er_lot,2eme_lot,Surface_Carrez_du_2eme_lot,3eme_lot,Surface_Carrez_du_3eme_lot,4eme_lot,Surface_Carrez_du_4eme_lot,5eme_lot,Surface_Carrez_du_5eme_lot,Nombre_de_lots,Type_local,Identifiant_local,Surface_reelle_bati,Nombre_pieces_principales,Nature_culture,Nature_culture_speciale,Surface_terrain,ID
0,03/01/2022,Vente,13.00,NaN,RUE,2280,DE LA LIBERTE,1000.00,BOURG-EN-BRESSE,1,53,NaN,AM,102,NaN,7.0,"24,10",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,Appartement,NaN,24.00,1.00,NaN,NaN,NaN,1
1,03/01/2022,Vente,98.00,NaN,RTE,0055,DE LA DOMBES,1480.00,SAVIGNEUX,1,398,NaN,ZE,187,NaN,1.0,"123,23",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,Appartement,NaN,140.00,3.00,NaN,NaN,NaN,2
2,06/01/2022,Vente,282.00,NaN,RTE,0130,DE POISATON,1560.00,MANTENAY-MONTLIN,1,230,NaN,ZM,124,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,Maison,NaN,108.00,5.00,S,NaN,649.00,3
3,05/01/2022,Vente,7.00,NaN,RUE,0031,DU CORNIER,1150.00,VAUX-EN-BUGEY,1,431,NaN,A,1613,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,Maison,NaN,85.00,4.00,S,NaN,310.00,4
4,06/01/2022,Vente,7.00,NaN,RUE,3125,DES PINS,1000.00,BOURG-EN-BRESSE,1,53,NaN,CS,218,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,99.00,5.00,S,NaN,765.00,5


In [153]:

colonnes_a_imputer = ["Surface_reelle_bati","Nombre_pieces_principales"]


sub_2 = sub[colonnes_a_imputer]
sub_2_imputed = imputer.transform(sub_2)

sub_1 = sub.drop(columns=colonnes_a_imputer)


# Créer un DataFrame à partir du tableau NumPy imputé
sub_2_imputed_df = pd.DataFrame(sub_2_imputed, columns=colonnes_a_imputer)


sub_final = pd.concat([sub_1, sub_2_imputed_df], axis=1)
sub_final.head()

,Date_mutation,Nature_mutation,No_voie,B/T/Q,Type_de_voie,Code_voie,Voie,Code_postal,Commune,Code_departement,Code_commune,Prefixe_de_section,Section,No_plan,No_Volume,1er_lot,Surface_Carrez_du_1er_lot,2eme_lot,Surface_Carrez_du_2eme_lot,3eme_lot,Surface_Carrez_du_3eme_lot,4eme_lot,Surface_Carrez_du_4eme_lot,5eme_lot,Surface_Carrez_du_5eme_lot,Nombre_de_lots,Type_local,Identifiant_local,Nature_culture,Nature_culture_speciale,Surface_terrain,ID,Surface_reelle_bati,Nombre_pieces_principales
0,03/01/2022,Vente,13.00,NaN,RUE,2280,DE LA LIBERTE,1000.00,BOURG-EN-BRESSE,1,53,NaN,AM,102,NaN,7.0,"24,10",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,Appartement,NaN,NaN,NaN,NaN,1,24.00,1.00
1,03/01/2022,Vente,98.00,NaN,RTE,0055,DE LA DOMBES,1480.00,SAVIGNEUX,1,398,NaN,ZE,187,NaN,1.0,"123,23",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,Appartement,NaN,NaN,NaN,NaN,2,140.00,3.00
2,06/01/2022,Vente,282.00,NaN,RTE,0130,DE POISATON,1560.00,MANTENAY-MONTLIN,1,230,NaN,ZM,124,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,Maison,NaN,S,NaN,649.00,3,108.00,5.00
3,05/01/2022,Vente,7.00,NaN,RUE,0031,DU CORNIER,1150.00,VAUX-EN-BUGEY,1,431,NaN,A,1613,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,Maison,NaN,S,NaN,310.00,4,85.00,4.00
4,06/01/2022,Vente,7.00,NaN,RUE,3125,DES PINS,1000.00,BOURG-EN-BRESSE,1,53,NaN,CS,218,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,S,NaN,765.00,5,99.00,5.00


In [154]:
sub_final['Type_local'].value_counts()

# recoder ....

Type_local
Maison                                      193425
Appartement                                  61572
Dépendance                                   46331
Local industriel. commercial ou assimilé     26713
Name: count, dtype: int64

In [ ]:
# Faire classif sur Code Type Local....

######

In [155]:
# Faire prédictions .....


def faire_prediction(row):
    if row['Type_local'] == "Maison":
        return model_reg_1.predict(row)[0]
    elif row['Type_local'] == "Appartement":
        return model_reg_2.predict(row)[0]
    # Ajoutez d'autres conditions pour les autres valeurs de 'Code_type_local' et les modèles associés si nécessaire
    elif row['Type_local'] == "Dépendance":
        return model_reg_3.predict(row)[0]
    # Ajoutez d'autres conditions pour les autres valeurs de 'Code_type_local' et les modèles associés si nécessaire
    elif row['Type_local'] == "Local industriel. commercial ou assimilé":
        return model_reg_4.predict(row)[0]
    else:
        pass

# Appliquez la fonction faire_prediction à chaque ligne du DataFrame et ajoutez les résultats à une nouvelle colonne 'TARGET'
sub_final['TARGET'] = sub_final.apply(faire_prediction, axis=1)
sub_final.head()

c:\Python311\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but Ridge was fitted with feature names
  warnings.warn(


ValueError: could not convert string to float: '03/01/2022'

In [ ]:
# recoder Type Local en Code type local 
# enlever année et mois dans la régression